In [ ]:
## Pro

## Problem 3

### Force vector derivation

Assuming no body forces ($[{f}^{b}] = [0]$)

$$
\begin{align*}
[{f}] & = \int_{\Gamma} [{N}]^{T} [{f}]^{s} d\Gamma \\
& = t_{z} \int_{y} [{N}]^{T} [{f}]^{s} dy
\end{align*}
$$

Surface traction is along the $\eta_{1} = 1$ in isoparametric space, thus

$$
\begin{align*}
[{f}] & = t_{z} \int_{y} [{N}]^{T} [{f}]^{s} dy \\
& = t_{z}*det([J^{\Gamma}])* \int_{-1}^{1} [{N}]^{T} [{f}]^{s} d\eta_{2}
\end{align*}
$$

Evaluating shape functions @ $\eta_{1} = 1$

$$
\begin{align*}
N_{1} & = 0.25*(\eta_{1} - 1)*(\eta_{2} - 1) \rightarrow 0.25*(1 - 1)*(\eta_{2} - 1) = 0 \\
N_{2} & = -0.25*(\eta_{1} + 1)*(\eta_{2} - 1) \rightarrow -0.25*(1 + 1)*(\eta_{2} - 1) = -0.5*(\eta_{2} - 1) \\
N_{3} & = 0.25*(\eta_{1} + 1)*(\eta_{2} + 1) \rightarrow 0.25*(1 + 1)*(\eta_{2} + 1) = 0.5*(\eta_{2} + 1) \\
N_{4} & = -0.25*(\eta_{1} - 1)*(\eta_{2} + 1) \rightarrow -0.25*(1 - 1)*(\eta_{2} + 1) = 0 \\
\end{align*}
$$

Assembling into the shape function matrix:

$$
[N] = \begin{bmatrix}
    N_{1} & 0 & N_{2} & 0 & N_{3} & 0 & N_{4} & 0 \\
    0 & N_{1} & 0 & N_{2} & 0 & N_{3} & 0 & N_{4}
\end{bmatrix} = \begin{bmatrix}
    0 & 0 & -0.5*(\eta_{2} - 1) & 0 & 0.5*(\eta_{2} + 1) & 0 & 0 & 0 \\
    0 & 0 & 0 & -0.5*(\eta_{2} - 1) & 0 & 0.5*(\eta_{2} + 1) & 0 & 0
\end{bmatrix}
$$

For the given surface traction, $det([J^{\Gamma}])$ requires components Jacobian components $J_{21}, J_{22}$. Hence, from the equation for the Jacobian in column format, only rows 2 and 4 of the shape function derivative matrix are required to compute $det([J^{\Gamma}])$. Therefore, only the derivatives with respect to $\eta_2$ of the shape functions can be inspected here.

Evaluating shape function derivatives @ $\eta_{1} = 1$

$$
\begin{align*}
\frac{\partial{N_{1}}}{\partial{\eta_{2}}} & = 0.25*(eta_1 - 1) \rightarrow 0.25*(1 - 1) = 0\\
\frac{\partial{N_{2}}}{\partial{\eta_{2}}} & = -0.25*(eta_1 + 1) \rightarrow -0.25*(1 + 1) = -0.5\\
\frac{\partial{N_{3}}}{\partial{\eta_{2}}} & = 0.25*(eta_1 + 1) \rightarrow 0.25*(1 + 1) = 0.5 \\
\frac{\partial{N_{4}}}{\partial{\eta_{2}}} & = -0.25*(eta_1 - 1) \rightarrow -0.25*(1 - 1) = 0 \\
\end{align*}
$$

Now computing Jacobian components $J_{21}$ and $J_{22}$:

$$
[J_{col}] = \frac{\partial[{N}]}{\partial{\mathbf{\eta}}}{[\hat{x}]}
$$

$$
J_{21} = \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{11}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{21}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{31}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{41}}
$$